In [ ]:
import json
with open('schulen.json') as f:
    schulen = json.load(f)

In [ ]:
import osmnx as ox
# get the building footprints in Macclesfield
place = "Landkreis Breisgau-Hochschwarzwald"
place = "Stadt Freiburg"
#tags = {'building': 'school'}
# Ebene Landkreis
tags = {'admin_level': '6'}
#gdf = ox.geometries_from_place(place, tags)

#tags = {'boundary': 'administrative'}   

bh_osmid='R1946367'
fr_osmid='R62768'
lk_bh=ox.geocode_to_gdf(fr_osmid, by_osmid=True)
lk_bh_poly=lk_bh.geometry[0]

In [ ]:
# Ebene Gemeinde
tags = {'admin_level': '8'}
gdf =ox.features.features_from_polygon(lk_bh_poly,tags)

In [ ]:
in_bh=gdf.within(lk_bh_poly)
gem_in_bh_gdf=gdf[in_bh & (gdf['name'].notnull()) & (gdf['type'] == 'boundary') ]  

In [ ]:
lk_bh

In [ ]:
eki=gem_in_bh_gdf[gem_in_bh_gdf["name"]=='Ehrenkirchen']

In [ ]:
eki.plot()

In [ ]:
## Findet alle Teilorte
#in_bh_gdf.plot()
tags = {'boundary': 'administrative'}
gem_bh_gdf =ox.features.features_from_polygon(eki_poly,tags)
in_bh=gem_bh_gdf.within(eki_poly)
ort_in_eki_gdf=gem_bh_gdf[in_bh & (gem_bh_gdf['name'].notnull()) & (gem_bh_gdf['type'] == 'boundary') & gem_bh_gdf['de:regionalschluessel'].isnull()] 


In [ ]:
figure, axes = ox.plot_graph(
    gdf_eki_walk,
    #ax=axe,
    #node_color=nc,
    #node_size=ns,
    node_alpha=0.8,
    edge_linewidth=0.2,
    edge_color="#999999",
    show=False, 
    close=False
)

In [ ]:
ort_in_eki_gdf

In [ ]:
type(gdf_eki_build)

In [ ]:
gem_in_bh_gdf

In [ ]:
from shapely.geometry import Point
import networkx as nx
node_colors = {}
node_time = {}
for cur in (schulen["schools"]):
    p=Point(cur["lng"],cur["lat"])
    #if p.within(eki_poly).any(): 
    if p.within(eki_poly): 
        poi=ox.distance.nearest_nodes(gdf_eki_walk, cur["lng"],cur["lat"])
        for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
            subgraph = nx.ego_graph(gdf_eki_walk, poi, radius=trip_time, distance="time")
            for node in subgraph.nodes():
                col_old=0
                if node in node_colors:
                    col_old=sum(node_colors[node])
                if sum(color)>col_old:
                    node_colors[node] = color
                col_time=10000000
                if node in node_time:
                    col_time=node_time[node]
                if trip_time<col_time:
                    node_time[node] = trip_time                    

In [ ]:
import pandas as pd
import numpy
from shapely.geometry import Point
import networkx as nx
import matplotlib.pyplot as plt
import geopandas as gpd

network_type = "walk"
trip_times = [5, 10, 15, 20, 25,30,40]  # in minutes
travel_speed = 3  # walking speed in km/hour
# get one color for each isochrone
iso_colors = ox.plot.get_colors(n=len(trip_times), cmap="plasma", start=0)
cmap = matplotlib.cm.cool
norm = matplotlib.colors.BoundaryNorm(trip_times, cmap.N)

sc=[]
for cur in (schulen["schools"]):
    p=Point(cur["lng"],cur["lat"])
    sc=sc+[[p,cur["name"]]]

if len(gem_in_bh_gdf)==0:
    gem_in_bh_gdf=lk_bh

alle_gem=pd.DataFrame()
for index_gem,row_gem in gem_in_bh_gdf.iterrows():
    gem_name=row_gem['name']
    #if gem_name=='Merdingen':
    if 1==1:
    
        gem_poly=row_gem.geometry
    
        tags = {'landuse':['residential','farmyard','industrial','commercial']}
        gdf_gem_lu =ox.features.features_from_polygon(gem_poly,tags)
        #tags = {'building': True}  
        #gdf_gem_build=ox.features_from_polygon(gem_poly,tags)
        tags = {'addr:housenumber': True}
        gdf_gem_hn=ox.features_from_polygon(gem_poly,tags)
        gdf_gem_hn['triptime']=120
        
        gdf_gem_walk=ox.graph_from_polygon(gem_poly,network_type=network_type)
        
        # add an edge attribute for time in minutes required to traverse each edge
        meters_per_minute = travel_speed * 1000 / 60  # km per hour to m per minute
        for _, _, _, data in gdf_gem_walk.edges(data=True, keys=True):
            data["time"] = data["length"] / meters_per_minute
    
        tags = {'boundary': 'administrative'}
        gem_bh_gdf =ox.features.features_from_polygon(gem_poly,tags)
        in_bh=gem_bh_gdf.within(gem_poly)
        ort_in_gem_gdf=gem_bh_gdf[in_bh & (gem_bh_gdf['name'].notnull()) & (gem_bh_gdf['type'] == 'boundary') & gem_bh_gdf['de:regionalschluessel'].isnull()] 

        t=pd.DataFrame({'GemName':[gem_name]})
        x=ort_in_gem_gdf['name'].values
        if gem_name not in x:
            x=numpy.append(x,[[gem_name]])
        ort=pd.DataFrame({'OrtName': x})
        #ort=pd.DataFrame[{'Name':x[0]}]
        y=numpy.unique(gdf_gem_lu['landuse'].values)
        y=numpy.append(y,[['unbekannt']])
        #y.append('unbekannt')
        landuse_df=pd.DataFrame({'Landuse': y})
        tt=pd.DataFrame({'WegeDauer':[0]+trip_times+[120]})
        
        res=pd.merge(ort,landuse_df, how='cross')
        res2=pd.merge(t,res, how='cross')
        boxen=pd.merge(res2,tt, how='cross')
        boxen=boxen.set_index(['GemName','OrtName','Landuse','WegeDauer'])
        boxen['Anzahl']=0
    
        
        
        node_colors = {}
        node_time = {}
        hn_colors = {}
        for cur in (schulen["schools"]):
            p=Point(cur["lng"],cur["lat"])
            if p.within(gem_poly): 
                poi=ox.distance.nearest_nodes(gdf_gem_walk, cur["lng"],cur["lat"])
                for trip_time, color in zip(sorted(trip_times, reverse=True), iso_colors):
                    subgraph = nx.ego_graph(gdf_gem_walk, poi, radius=trip_time, distance="time")
                    for node in subgraph.nodes():
                        col_old=0
                        if node in node_colors:
                            col_old=sum(node_colors[node])
                        if sum(color)>col_old:
                            node_colors[node] = color
                        col_time=10000000
                        if node in node_time:
                            col_time=node_time[node]
                        if trip_time<col_time:
                            node_time[node] = trip_time    
        for idx_build, row_build in gdf_gem_hn.iterrows():
            poly_build=row_build['geometry']
            #poly_ort = gpd.GeoSeries(row_orte['geometry'])
            p_center=poly_build.centroid
            poi=ox.distance.nearest_nodes(gdf_gem_walk, p_center.x,p_center.y)
            WegeDauer=120
            if poi in node_time:
                WegeDauer=node_time[poi]
            if poi in node_time:
                hn_colors[idx_build[1]]=node_colors[poi]
                gdf_gem_hn.loc[idx_build,['triptime']]=node_time[poi]
            
            Landuse='unbekannt'
            for idx_landuse, row_landuse in gdf_gem_lu.iterrows():
                lu_type=row_landuse['landuse']
                poly_lu= gpd.GeoSeries(row_landuse['geometry'])
                if poly_lu.contains(p_center).any():
                    Landuse=lu_type
            Ortsteil=gem_name
            for idx_ort, row_ort in ort_in_gem_gdf.iterrows():
                poly_ortt = gpd.GeoSeries(row_ort['geometry'])
                if poly_ortt.contains(p_center).any():
                    Ortsteil=row_ort['name']    
            boxen.at[(gem_name,Ortsteil,Landuse,WegeDauer),'Anzahl']=boxen.at[(gem_name,Ortsteil,Landuse,WegeDauer),'Anzahl']+1
    
        boxen.to_csv('erstellte_daten/'+gem_name+'.csv',index=True)
        if len(alle_gem)<5:
            alle_gem=boxen
        else:
            alle_gem=alle_gem+boxen
        alle_gem.to_csv('erstellte_daten/'+place+'.csv',index=True)

        br=1
        anz=1

        gp_b=gem_poly.bounds[2]-gem_poly.bounds[0]
        gp_l=gem_poly.bounds[3]-gem_poly.bounds[1]
        md=max(gp_b,gp_l)
        size=40/md



     
        fig, axs = plt.subplots(anz,br,figsize=(int(gp_b*size), int(gp_l*size)))
        #fig, axs = plt.subplots(anz,br)
        axs.set_title(gem_name)
        for idx_orte, row_orte in ort_in_gem_gdf.iterrows():
            poly_ort = gpd.GeoSeries(row_orte['geometry'])
            p_center=poly_ort.centroid
            if gem_poly.contains(p_center).any():
                poly_ort.plot(ax=axs,facecolor='none', edgecolor='green')
                axs.annotate(text=row_orte["name"], xy=(p_center[0].x,p_center[0].y), horizontalalignment='center')
        if len(ort_in_gem_gdf)==0:
            gem_poly_gpd = gpd.GeoSeries(gem_poly)
            gem_poly_gpd.plot(ax=axs,facecolor='none', edgecolor='blue')    
        gdf_gem_hn.plot(ax=axs,column='triptime',markersize =5, cmap=cmap,norm=norm)
        #nc = [node_colors[node] if node in node_colors else "none" for node in gdf_gem_walk.nodes()]
        #ns = [5 if node in node_colors else 0 for node in gdf_gem_walk.nodes()]
        #poly_ort.plot(ax=axs,facecolor='none', edgecolor='green')
        fig, ax = ox.plot_graph(
            gdf_gem_walk,
            ax=axs,
            #node_color=nc,
            #node_size=ns,
            node_alpha=0.8,
            edge_linewidth=0.2,
            edge_color="#999999",
            show=False, 
            close=False
        )
        for cur in sc:
            axs.scatter(cur[0].x,cur[0].y,color='red',s=80)    
            axs.annotate(text=cur[1], xy=(cur[0].x,cur[0].y), horizontalalignment='center')
            #axs[sw//br,sw%br].set_ylim(poly_gem.bounds["miny"][0],poly_gem.bounds["maxy"][0])
            #axs[sw//br,sw%br].set_xlim(poly_gem.bounds["minx"][0],poly_gem.bounds["maxx"][0])
        plt.savefig('images/'+gem_name+'.png')
        #plt.show()



In [ ]:
poly_gem_gpd.plot()

In [ ]:
import networkx as nx
from shapely.geometry import Point
import matplotlib.pyplot as plt

trip_times = [5, 10, 15, 20, 25,30,40]
cmap = matplotlib.cm.cool
norm = matplotlib.colors.BoundaryNorm(trip_times, cmap.N)



br=1
anz=1
fig, axs = plt.subplots(anz,br,figsize=(30, 120))
sw=0
poly_gem = gem_poly
axs.set_title(gem_name)
for idx_orte, row_orte in ort_in_gem_gdf.iterrows():
    poly_ort = gpd.GeoSeries(row_orte['geometry'])
    p_center=poly_ort.centroid
    if poly_gem.contains(p_center).any():
        poly_ort.plot(ax=axs,facecolor='none', edgecolor='green')
        axs.annotate(text=row_orte["name"], xy=(p_center[0].x,p_center[0].y), horizontalalignment='center')
    gdf_gem_hn.plot(ax=axs,column='triptime',markersize =5, cmap=cmap,norm=norm)
    nc = [node_colors[node] if node in node_colors else "none" for node in gdf_gem_walk.nodes()]
    ns = [5 if node in node_colors else 0 for node in gdf_gem_walk.nodes()]
    poly_ort.plot(ax=axs,facecolor='none', edgecolor='green')
    fig, ax = ox.plot_graph(
        gdf_gem_walk,
        ax=axs,
        #node_color=nc,
        #node_size=ns,
        node_alpha=0.8,
        edge_linewidth=0.2,
        edge_color="#999999",
        show=False, 
        close=False
    )
    for cur in sc:
        axs.scatter(cur.x,cur.y,color='red',s=80)
    #axs[sw//br,sw%br].set_ylim(poly_gem.bounds["miny"][0],poly_gem.bounds["maxy"][0])
    #axs[sw//br,sw%br].set_xlim(poly_gem.bounds["minx"][0],poly_gem.bounds["maxx"][0])
#plt.savefig('images/'+gem_name+'.png')
plt.show()

In [ ]:
#hn_colors_series = pd.DataFrame.from_dict(hn_colors,orient='index')
#hn_colors_series

colors = map(lambda x: hn_colors.get(x, (0.5,0.5,0.5,1)), gdf_gem_hn.index.get_level_values(1))

In [ ]:
#hn_colors
for cur in gdf_gem_hn.index:
    osmid =cur[1] 
    #print (cur[1])
    if osmid in hn_colors:
        print(hn_colors[osmid])

In [ ]:
idx_build.index

In [ ]:
hc = [hn_colors[node] if node in hn_colors else "none" for node in hn_colors.nodes()]


In [ ]:
gdf_gem_hn.loc[idx_build,['triptime']]=15

In [ ]:
gdf_gem_hn.loc[idx_build,['triptime']]=node_time[poi]

In [ ]:
gdf_gem_hn['tripkat']

In [ ]:
        gp_b=gem_poly.bounds[2]-gem_poly.bounds[0]
        gp_l=gem_poly.bounds[3]-gem_poly.bounds[1]
        md=max(gp_b,gp_l)
        size=40/md
int(gp_b*size)
int(gp_l*size)

In [ ]:
sc=[]
for cur in (schulen["schools"]):
    p=Point(cur["lng"],cur["lat"])
    #if p.within(eki_poly).any(): 
    if p.within(gem_poly): 
        p=Point(cur["lng"],cur["lat"])
        sc=sc+[[p,cur["name"]]]
sc        

In [ ]:
for cur in sc:
    print(cur[0].x,cur[1])

In [ ]:
import matplotlib
matplotlib.colors.rgb_to_hsv([0.050383, 0.029803, 0.527975])

In [ ]:
max((gem_poly.bounds[2]-gem_poly.bounds[0]),(gem_poly.bounds[3]-gem_poly.bounds[1]))